# Contexte 
Le Problème du Voyageur de Commerce (TSP) pose la question suivante : « Étant donnée une liste de
villes et les distances entre chaque paire de villes, quel est le plus court trajet possible qui visite chaque
ville et revient à la ville d’origine ? » Il s’agit d’un problème N P-difficile en optimisation combinatoire,
important en recherche opérationnelle et en informatique théorique.
Le problème a été formulé pour la première fois en 1930 et est l’un des problèmes les plus étudiés
en optimisation. Il est utilisé comme benchmark pour de nombreuses méthodes d’optimisation. Bien que
le problème soit computationnellement difficile, de nombreuses heuristiques et algorithmes exacts sont
connus, de sorte que certaines instances avec des dizaines de milliers de villes peuvent être résolues com-
plètement et même des problèmes avec des millions de villes peuvent être approximés à une fraction de
1%.
Formellement, soit G = (V,E,w) un graphe complet non orienté tel que |V| = n et w : E → R≥0
la fonction de poids. C’est-à-dire que G est le graphe Kn avec des poids sur ses arêtes. Dans ce modèle,
chaque sommet représente une ville et chaque arête pondérée reliant deux sommets représente la distance
entre les villes correspondantes.
Ainsi, étant donné un graphe complet pondéré non orienté, le TSP consiste à trouver un cycle ha-
miltonien (un cycle qui traverse chaque sommet exactement une fois) qui minimise la somme de ses
poids.

# Fonctions

In [2]:
def get_data(path : str) -> list:
    """
    get_data("Projet/17.in")
    """
    D = []
    with open(path, "r") as f:
        N  = int(f.readline())

        for _ in range(N):
            D.append(list(map(int, f.readline().split())))
    return D

# Question 1
Décrire (d’autres) situations réelles qui peuvent être modélisées comme un TSP.
- Un ensemble de mots est donné. Le but est de passer par chaque mot en éffectuant le moins d'action possible. Chaque mot peut se transformer en un autre en :
    - changeant une lettre
    - ajoutant une lettre
    - enlevant une lettre 
- Un telescope doit observer un certain nombre de planetes. Le changement de possition de la lunette est couteux. Le but est de prévoir le chemin le plus court passant par toutes les planètes, afin de réduire les coûts.
- Un personnage sur Amongus doit effectuer toutes ses tâches. Etant donné la présence d'imposteurs dans le groupe, il doit effectuer ses tâches le plus vite possible

# Question 2
Développer et implémenter un algorithme exact pour résoudre le TSP, utilisant la technique du
Branch and Bound.

In [ ]:
from math import inf

class Solution :
    def __init__(self, vars, fixed_vars) :
        # x[i][j] = 0/1
        self.vars = [[0] for i in range()]
        self.fixed_vars = fixed_vars
        self.left = None
        self.right = None

    def calculateSol(self, data):
        result = 0
        for var in self.vars :
            result += 
        return result

    def fix_var(self,i,j,value) : 
        self.fixed_vars.append([i,j,value])

    def copy(self) -> Solution :
        var_copy = []
        for var in self.vars : 
            var_copy.append(var)
        return Solution(var_copy)
    
    def addLeft(self, solution):
        self.left = solution

    def addRight(self, solution):
        self.right = solution

    def verify_cycle_hamiltonien():
        # Vérifier qu'on peut aller de chaque node à chaque node en un nombre fini d'étape
        pass

class Arbre :
    def __init__(self) :
        self.first = None,

    def addFirst():
        pass

    def findBestLeaf() -> Solution :
        pass

In [ ]:
import pulp

def solve_lp(distances, fixed={}):
    n = len(distances)
    prob = pulp.LpProblem("TSP_LP", pulp.LpMinimize)

    x = {
        (i, j): pulp.LpVariable(f"x_{i}_{j}", 0, 1)
        for i in range(n) for j in range(n) if i != j
    }

    # Objective
    prob += pulp.lpSum(distances[i][j] * x[i, j] for i, j in x)

    # Degree constraints
    for i in range(n):
        prob += pulp.lpSum(x[i, j] for j in range(n) if i != j) == 1
        prob += pulp.lpSum(x[j, i] for j in range(n) if i != j) == 1

    # Fixed variables (branching)
    for (i, j), val in fixed.items():
        prob += x[i, j] == val

    prob.solve(pulp.PULP_CBC_CMD(msg=0))

    return prob, x

def branch_and_bound(distances):
    best_cost = float("inf")
    best_sol = None

    stack = [ {} ]  # fixed variables

    while stack:
        fixed = stack.pop()
        prob, x = solve_lp(distances, fixed)

        if prob.status != 1:
            continue

        value = pulp.value(prob.objective)
        if value >= best_cost:
            continue

        # Check integrality
        fractional = None
        for (i, j), var in x.items():
            v = var.value()
            if v > 1e-6 and v < 1 - 1e-6:
                fractional = (i, j)
                break

        if fractional is None:
            best_cost = value
            best_sol = x
            continue

        i, j = fractional
        stack.append({**fixed, (i, j): 0})
        stack.append({**fixed, (i, j): 1})

    return best_cost

def find_subtours(x, n):
    visited = [False] * n
    subtours = []

    adj = {i: [] for i in range(n)}
    for (i, j), var in x.items():
        if var.value() > 0.5:
            adj[i].append(j)

    for i in range(n):
        if not visited[i]:
            stack = [i]
            component = []
            while stack:
                u = stack.pop()
                if visited[u]:
                    continue
                visited[u] = True
                component.append(u)
                for v in adj[u]:
                    if not visited[v]:
                        stack.append(v)
            if len(component) < n:
                subtours.append(component)

    return subtours

def solve_lp_with_cuts(distances, fixed):
    n = len(distances)
    prob = pulp.LpProblem("TSP_LP", pulp.LpMinimize)

    x = {
        (i, j): pulp.LpVariable(f"x_{i}_{j}", 0, 1)
        for i in range(n) for j in range(n) if i != j
    }

    # Objective
    prob += pulp.lpSum(distances[i][j] * x[i, j] for i, j in x)

    # Degree constraints
    for i in range(n):
        prob += pulp.lpSum(x[i, j] for j in range(n) if i != j) == 1
        prob += pulp.lpSum(x[j, i] for j in range(n) if i != j) == 1

    # Branching constraints
    for (i, j), val in fixed.items():
        prob += x[i, j] == val

    # Cutting-plane loop
    while True:
        prob.solve(pulp.PULP_CBC_CMD(msg=0))
        if prob.status != 1:
            return None, None

        subtours = find_subtours(x, n)
        if not subtours:
            break

        for S in subtours:
            prob += (
                pulp.lpSum(x[i, j] for i in S for j in S if i != j)
                <= len(S) - 1
            )

    return prob, x

def branch_and_cut(distances):
    best_cost = float("inf")
    best_solution = None

    stack = [ {} ]  # branching constraints

    while stack:
        fixed = stack.pop()
        prob, x = solve_lp_with_cuts(distances, fixed)

        if prob is None:
            continue

        value = pulp.value(prob.objective)
        if value >= best_cost:
            continue

        # Check integrality
        fractional = None
        for (i, j), var in x.items():
            v = var.value()
            if 1e-6 < v < 1 - 1e-6:
                fractional = (i, j)
                break

        if fractional is None:
            best_cost = value
            best_solution = x
            continue

        # Branch
        i, j = fractional
        stack.append({**fixed, (i, j): 0})
        stack.append({**fixed, (i, j): 1})

    return best_cost, best_solution

def extract_tour(x, n, start=0):
    tour = [start]
    current = start

    while True:
        for j in range(n):
            if j != current and x[current, j].value() > 0.5:
                if j == start:
                    tour.append(start)
                    return tour
                tour.append(j)
                current = j
                break


In [ ]:
dist_matrix = get_data("Projet/100.in")
n = len(dist_matrix)

# Solve TSP using Branch-and-Cut
best_cost, best_solution = branch_and_cut(dist_matrix)

# Output results
print("Optimal cost:", best_cost)

tour = extract_tour(best_solution, n)
print("Optimal tour:", tour)

# QUESTION 3
Développer et implémenter une heuristique constructive pour résoudre le TSP.